### Матричный шифр

Предполагается, что буквы алфавита занумерованы и рассматриваются как элементы некоторого алгебраического кольца. Если к n-грамме сообщения применить матрицу $a_{ij}$, то получится n-грамма криптограммы:
\begin{equation}
l_i=\displaystyle\sum_{j=1}^{n}a_{ij}m_i, i=1...n
\end{equation}

Для расшифрования используется обратная матрица

\begin{equation}
A^{-1} = \frac{A^{*}}{\det(A)}
\end{equation}

где $A^{*}$ - присоединенная матрица, каждый элемент $a^{*}_{ij}$ которой является алгебраическим дополнением $a_{ij}$

In [59]:
al = [chr(i) for i in range(ord('А'), ord('Я') + 1) if i is not ord('Ё')]
print(al)

['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я']


In [2]:
import numpy as np # для реализации матричных операций будем использовать библиотеку NumPy
import numpy.linalg as linalg

In [4]:
A = np.matrix([[1,-2,3],[0,4,-1],[5,0,0]]) # инициализация матрицы
print(A)

[[ 1 -2  3]
 [ 0  4 -1]
 [ 5  0  0]]


Попробуем зашифровать первый блок:

In [5]:
key = np.matrix([[2,2,4,2,0], [1,4,3,4,2], [1,1,4,2,2], [0,3,4,2,4], [2,4,3,0,1]])
linalg.det(key) # удостоверимся, что существует обратная матрица

-78.0

In [27]:
block = [13,5,2,17,31] # нумерацию в алфавите ведем с нуля

In [7]:
np.matrix(block).swapaxes(0,1) # преобразуем список в np-вектор

matrix([[13],
        [ 5],
        [ 2],
        [17],
        [31]])

In [8]:
cipher = key * np.matrix(block).swapaxes(0,1)
cipher

matrix([[ 78],
        [169],
        [122],
        [181],
        [ 83]])

In [9]:
res = linalg.inv(key) * cipher # умножим обратную матрицу на блок шифротекста
res = np.matrix.round(res,0).astype(int) # приведем результат к целому типу
res = res.squeeze().tolist()[0] # преобразуем вектор обратно к списку
res

[13, 5, 2, 17, 31]

In [10]:
[al[i] for i in res] # по индексу символа в алфавите восстановим блок исходного сообщения

['Н', 'Е', 'В', 'С', 'Я']

In [37]:
linalg.inv(key)

array([[-0.4,  0.2,  0.4],
       [ 0.7, -0.6, -0.2],
       [ 0.2,  0.4, -0.2]])

In [38]:
linalg.det(key)

10.000000000000002

Добавим вспомогательные функции:

In [3]:
import requests
from bs4 import BeautifulSoup as bs

def clear_paragraphs(ps:list, start=14, stop=16):
    return ''.join([i.text for i in ps[start:stop]]).replace('\xad', '').replace('\xa0', '')

def test_text(url='https://spacemorgue.com/matter-matters/'):
    '''
    Возвращает тестовый текст длиной 2408 символов
    '''
    soup = bs(requests.get(url).text, 'html.parser')
    return clear_paragraphs(soup.select_one('article').find_all('p'))

def prepare(text, specials={',':'ЗПТ','.':'ТЧК', ' ':'', '—':'ТИРЕ', '«':'КВЧ', '»':'КВЧ'}):
    '''
    Возвращает текст, удаляя пробельные символы в тексте и заменяя пунктуацию телеграфными обозначениями
    '''
    for i in specials.keys():
        text = text.replace(i, specials[i])
    return text

def comp(text, specials={'ЗПТ':',','ТЧК':'.','ТИРЕ':'—','КВЧ':'"'}):
    '''
    Возвращает текст, заменяя телеграфные обозначения пунктуацией
    '''
    for i in specials.keys():
        text = text.replace(i, specials[i])
    return text

In [4]:
def equivalent(text, al):
    '''
    Возвращает список чисел - индексов символов в данном алфавите
    '''
    eq = list()
    for i in prepare(text):
        eq.append(al.index(i))
    return eq

def separate(text, block_size, void_ch=None):
    '''
    Делит список числовых эквивалентов на блоки заданной длины. 
    При наличии неполного блока заполняет его пустыми символами.
        
        Параметры:
            text (list): Список числовых эквивалентов
            block_size (int): Размер блока
            void_ch (int): Индекс пустого символа на данном алфавите
        Возвращает:
            vectors (list): Список, состоящий из списков фиксированной длины
    '''
    vectors = list()
    
    if len(text) % block_size: # недостает символов для заполнения последнего блока
        void_count = block_size - (len(text) % block_size) # количество пустых символов
        text = text + [len(al) - 1] * void_count
            
    for i in range(0, len(text), block_size):
        block = text[i:i+block_size]
        vectors.append(block)
        assert len(vectors[-1]) == block_size, f'Created incomplete block - {vectors[-1]}'
    return vectors

Инициализируем алфавит:

In [13]:
al = [chr(i) for i in range(ord('А'), ord('Я') + 1) if i is not ord('Ё')]
al.append('0') # добавим пустой символ для заполнения неполных блоков 
print(al)

['А', 'Б', 'В', 'Г', 'Д', 'Е', 'Ж', 'З', 'И', 'Й', 'К', 'Л', 'М', 'Н', 'О', 'П', 'Р', 'С', 'Т', 'У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Ь', 'Э', 'Ю', 'Я', '0']


Наконец, реализуем функцию шифрования:

In [5]:
def matrix_encrypt(plaintext, al, key, block_size=5):
    assert linalg.det(key) != 0, 'Given key is not an invertible matrix'
    ciphertext = list()
    for block in separate(equivalent(plaintext, al), block_size):
        vector = key * np.matrix(block).swapaxes(0,1)
        ciphertext.extend(vector.squeeze().tolist()[0])
    return ' '.join([str(i) for i in ciphertext])

Проверяем:

In [15]:
print('Матрица-ключ:', end='\n\n')
key = np.array([[2,2,4,2,0], [1,4,3,4,2], [1,1,4,2,2], [0,3,4,2,4], [2,4,3,0,1]])
print(key, end='\n\n')
print('Шифротекст:')
matrix_encrypt('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.', al, key)

Матрица-ключ:

[[2 2 4 2 0]
 [1 4 3 4 2]
 [1 1 4 2 2]
 [0 3 4 2 4]
 [2 4 3 0 1]]

Шифротекст:


'78 169 122 181 83 86 127 98 134 94 156 214 174 222 153 62 136 72 136 106 120 175 144 213 151 114 125 107 115 103 132 172 121 134 106 82 136 105 128 59 258 351 289 350 214'

In [11]:
def clean_void(text:list, al):
    return list(filter(lambda x: x != al[-1], text))

def matrix_decrypt(ciphertext, al, key, block_size=5):
    plaintext = list()
    for block in separate(ciphertext.split(), block_size):
        vector = np.matrix([int(i) for i in block]).swapaxes(0,1)
        res = linalg.inv(key) * vector
        res = np.matrix.round(res,0).astype(int)
        res = res.squeeze().tolist()[0]
        plaintext.extend([al[i] for i in res])
    return comp(''.join(clean_void(plaintext, al)))

print('Матрица-ключ:', end='\n\n')
print(key, end='\n\n')

#matrix_decrypt(matrix_encrypt('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.', al, key),al,key)

Матрица-ключ:

[[2 2 2]
 [1 0 2]
 [4 2 1]]



In [36]:
al.index('К')

10

In [9]:
print('Матрица-ключ:', end='\n\n')
key = np.array([[2,2,2],[1,0,2],[4,2,1]])
print(key, end='\n\n')
print('Шифротекст:')
matrix_encrypt('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.', al, key, block_size=3)

Матрица-ключ:

[[2 2 2]
 [1 0 2]
 [4 2 1]]

Шифротекст:


'40 17 64 116 37 140 48 22 57 86 35 106 110 64 123 52 8 68 60 19 63 70 19 98 92 33 133 58 29 76 72 39 95 78 45 103 46 36 28 102 38 128'

In [12]:
matrix_decrypt(matrix_encrypt('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.', al, key, block_size=3),al,key,3)

'НЕВСКИЙ,КТОЧИТАЕТ,ВЧТЕНИИСИЛУЗНАЕТ.'

In [17]:
print('Сгенерируем псевдослучайный ключ:', end='\n\n')
key = np.matrix(np.random.randint(5, size=(5,5)))
print(key, end='\n\n')
print(f'Шифротекст:\n\n{matrix_encrypt("НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.", al, key)}\n\n')
print(f'Расшифровка:\n\n{matrix_decrypt(matrix_encrypt("НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.", al, key),al,key)}')

Сгенерируем псевдослучайный ключ:

[[2 1 3 3 1]
 [4 0 2 2 3]
 [1 0 0 4 2]
 [0 0 1 3 4]
 [1 3 2 4 4]]

Шифротекст:

119 183 143 177 224 91 117 68 90 140 165 205 120 152 232 67 96 64 87 154 112 137 61 116 188 112 138 66 69 123 139 149 107 96 167 113 152 121 131 167 280 316 215 256 373


Расшифровка:

НЕВСЯКИЙ,КТОЧИТАЕТ,ВЧТЕНИИСИЛУЗНАЕТ.


In [39]:
text = prepare(test_text().upper().replace('-', '—'))
ext = al.copy()
ext.extend(['X', 'I'])

In [41]:
print('Сгенерируем псевдослучайный ключ:', end='\n\n')
key = np.matrix(np.random.randint(5, size=(5,5)))
print(key, end='\n\n')
print(f'Шифротекст:\n\n{matrix_encrypt(text, ext, key)[:150]}...\n\n')
print(f'Расшифровка:\n\n{matrix_decrypt(matrix_encrypt(text, ext, key),ext,key)[:150]}...')

Сгенерируем псевдослучайный ключ:

[[2 1 4 2 4]
 [3 1 4 2 0]
 [0 3 1 3 3]
 [1 3 4 4 2]
 [3 0 4 1 0]]

Шифротекст:

143 104 89 130 91 195 76 133 146 67 177 113 123 167 96 213 129 162 209 104 191 72 154 158 55 156 139 149 203 100 201 142 158 212 112 161 181 112 191 1...


Расшифровка:

НЕЛИНЕЙНАЯИСТАТИСТИЧЕСКАЯПРИЧИННОСТЬСНОВАПРИВНОСЯТВФИЛОСОФСКУЮКОНЦЕПЦИЮКАУЗАЛЬНОЙСВЯЗИСЛОЖНОСТЬ,КОТОРАЯБЫЛАПОТЕРЯНАСПОЯВЛЕНИЕМПОНЯТИЯПОСТОЯННОЙКОНЪЮНК...


### Шифр Плэйфера

Шифр  основан  на лозунговом способе заполнения шифртаблицы. Для создания таблицы нужно заполнить пустые ячейки матрицы буквами ключевого слова, затем по порядку заполнить оставшиеся ячейки матрицы символами алфавита, не встречающимися в ключевом слове.

Для того чтобы зашифровать сообщение, необходимо разбить его на биграммы.

Переход  от  биграмм  входного  текста  к  биграммам  выходного  текста осуществляется по следующим правилам: 

1. Если буквы входной биграммы оказались в  одном  столбце  таблицы,  шифрование  происходит  сверху  вниз;  
2. Если  же  буквы входной  биграммы  оказались  в  одной  строке таблицы,  то  шифрование осуществляется слева направо, а расшифрование — наоборот. 
3. Если буквы биграммы совпадают, их  следует  разделить  буквой  с  наименьшей  частотой  встречаемости (например, Ф - для литературных текстов). 
4. Если буквы входной биграммы оказались в разных столбцах и строках таблицы, то рисуется воображаемый прямоугольник, а выходная биграмма берется как его альтернативные вершины

In [77]:
def gen_square(key_phrase, al, size=5):
    assert len(set(key_phrase)) == len(key_phrase) # в лозунге отсутствуют повторяющиеся символы
    al_plaifair = [i for i in al if i not in key_phrase] # убираем из алфавита символы, встречающиеся в лозунге
    key_phrase = list(key_phrase)

    square = list()
    
    for i in range(size):
        row = list()
        for i in range(size+1):
            if key_phrase:
                row.append(key_phrase.pop(0))
            else:
                row.append(al_plaifair.pop(0))
        square.append(row)
    return np.matrix(square)

In [80]:
al_plaifair = al.copy()

In [81]:
al_plaifair.remove('Й')
al_plaifair.remove('Ь')

In [95]:
square = gen_square('БЛИНДАЖ',al_plaifair)
square

matrix([['Б', 'Л', 'И', 'Н', 'Д', 'А'],
        ['Ж', 'В', 'Г', 'Е', 'З', 'К'],
        ['М', 'О', 'П', 'Р', 'С', 'Т'],
        ['У', 'Ф', 'Х', 'Ц', 'Ч', 'Ш'],
        ['Щ', 'Ъ', 'Ы', 'Э', 'Ю', 'Я']], dtype='<U1')

Попробуем найти альтернативные вершины прямоугольника, если буквы находятся в различных частях таблицы:

In [22]:
np.where(square == 'А')[1][0] # при помощи функции where мы можем получить индексы символа в матрице

5

In [23]:
angle_1 = np.where(square == 'Г')
angle_2 = np.where(square == 'Ч')

angle_1[0][0], angle_2[1][0], angle_2[0][0], angle_1[1][0]

(1, 4, 3, 2)

In [24]:
np.where(square == 'З'), np.where(square == 'Х')

((array([1], dtype=int64), array([4], dtype=int64)),
 (array([3], dtype=int64), array([2], dtype=int64)))

In [45]:
def avoid_double(text_l):
    '''
    Разделяет пары символов символом с наименьшей частотой встречаемости
    '''
    res = list()
    for pair in text_l:
        if pair[0] == pair[1]:
            pair = 'Ф'.join(pair)
        res.append(pair)
    return ''.join(res)

def separate_with(text, block_size, func=None):
    '''
    Делит список числовых эквивалентов на блоки заданной длины. 
    При наличии неполного блока заполняет его пустыми символами.
        
        Параметры:
            text (str): Список числовых эквивалентов
            block_size (int): Размер блока
            func (function): Функция, которая будет применена к тексту
        Возвращает:
            vectors (list): Список, состоящий из списков фиксированной длины
    '''
    vectors = list()
    if func:
        text = func(separate_with(text, 2))
            
    for i in range(0, len(text), block_size):
        block = text[i:i+block_size]
        if len(block) != block_size:
            block += 'Ф'
        vectors.append(block)
    return vectors

In [100]:
def plaifair_encrypt(plaintext, al, key_phrase):
    ciphertext = list()
    plaintext = plaintext.replace('Й', 'И').replace('Ё','Е').replace('Ь','Ъ')
    square = gen_square(key_phrase, al_plaifair)
    
    for bigram in separate_with((prepare(plaintext)), 2, avoid_double):
        # первый индекс - номер символа в биграмме, второй - координата, третий - ее значение (всегда 0)
        indexes = [np.where(square == bigram[0]), np.where(square == bigram[1])]

        assert bigram[0] in square, f'{bigram[0]} not in square'
        assert bigram[1] in square, f'{bigram[1]} not in square'
        in_a_column = indexes[0][1] == indexes[1][1]
        in_a_row = indexes[0][0] == indexes[1][0]

        if in_a_row:
            # сдвигаем символы слева-направо
            indexes_new = [(indexes[0][0][0], (indexes[0][1][0] + 1) % square.shape[1])]
            indexes_new.append((indexes[1][0][0], (indexes[1][1][0] + 1) % square.shape[1]))
        elif in_a_column:
            # сдвигаем символы сверху-вниз
            indexes_new = [((indexes[0][0][0] + 1) % square.shape[0], indexes[0][1][0])]
            indexes_new.append(((indexes[1][0][0] + 1) % square.shape[0], indexes[1][1][0]))
        else:
            # находим противоположные углы
            indexes_new = [(indexes[0][0][0], indexes[1][1][0])]
            indexes_new.append((indexes[1][0][0], indexes[0][1][0]))

        bigramm_new = [square.item(indexes_new[0])]
        bigramm_new.append(square.item(indexes_new[1]))
        ciphertext.append(''.join(bigramm_new))
    return ' '.join(ciphertext)

In [27]:
plaifair_encrypt('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.', al_plaifair,'БЛИНДАЖ')

'ЕР ЗО АТ ЛХ ДГ РМ ТШ СФ АП НК СК РМ ЗФ РК ДН ХЛ ПД БФ ЕД НК ОШ СШ ВШ'

In [88]:
def plaifair_decrypt(ciphertext, al, keyphrase):
    plaintext = list()
    square = gen_square(keyphrase, al_plaifair)
    
    for bigram in separate_with((prepare(ciphertext)), 2):
        indexes = [np.where(square == bigram[0]), np.where(square == bigram[1])]

        in_a_column = indexes[0][1] == indexes[1][1]
        in_a_row = indexes[0][0] == indexes[1][0]
        if in_a_row:
            indexes_new = [(indexes[0][0][0], (indexes[0][1][0] - 1) % square.shape[1])]
            indexes_new.append((indexes[1][0][0], (indexes[1][1][0] - 1) % square.shape[1]))
        elif in_a_column:

            indexes_new = [((indexes[0][0][0] - 1) % square.shape[0], indexes[0][1][0])]
            indexes_new.append(((indexes[1][0][0] - 1) % square.shape[0], indexes[1][1][0]))
        else:
            indexes_new = [(indexes[0][0][0], indexes[1][1][0])]
            indexes_new.append((indexes[1][0][0], indexes[0][1][0]))
        
        bigramm_new = [square.item(indexes_new[0])]
        bigramm_new.append(square.item(indexes_new[1]))
        plaintext.append(''.join(bigramm_new))
    return comp(''.join(plaintext))

In [89]:
plaifair_decrypt(plaifair_encrypt('НЕ ВСЯКИЙ, КТО ЧИТАЕТ, В ЧТЕНИИ СИЛУ ЗНАЕТ.', al_plaifair,'БЛИНДАЖ'), al_plaifair,'БЛИНДАЖ')

'НЕВСЯКИФИ,КТОЧИТАЕТ,ВЧТЕНИФИСИЛУЗНАЕТФ.Ф'

In [29]:
plaifair_decrypt('МС КЧ КЖ ОЖ БТ ЮО БЫ ФУ ВО ХВ АЦ ВР ИД ЧМ ЖЗ НЦ ЦЩ ЕС АФ ЕХ ФЪ', al_plaifair,'ВЫСТРЕЛ')

'КТОФОБЖЕГСЯНАСУПЕ,ДУЕТНАХОЛОДНУЮРЫБУ.Ф'

In [31]:
'ТОТ,КТОХОЧЕТЕСАТИХЬЦЭНГФМЖБСИНММИВЗСРТЯНМОЬВИЩЬТДНЖЕЯЧХЪ'[:15]

'ТОТ,КТОХОЧЕТЕСА'

In [32]:
len('ТОТ,КТОХОЧЕТЕСА')

15

In [35]:
''.join('СФ ЕК ФЕ ФВ ПЦ ЧС НЯ НЕ ВН ЛУ ДП ЩБ ВХ ЖЗ СЕ УА ЖЖ КА ЖЕ ВС БТ ЖЦ АГ УН ВЕ АБ ЗС СЫ ФЫ'.split())[:15]

'СФЕКФЕФВПЦЧСНЯН'

In [47]:
gen_square('СЕНТЯБРЬ', al_plaifair)

matrix([['С', 'Е', 'Н', 'Т', 'Я', 'Б'],
        ['Р', 'Ь', 'А', 'В', 'Г', 'Д'],
        ['Ж', 'З', 'И', 'К', 'Л', 'М'],
        ['О', 'П', 'У', 'Ф', 'Х', 'Ц'],
        ['Ч', 'Ш', 'Щ', 'Ъ', 'Ы', 'Э']], dtype='<U1')

In [64]:
plaifair_encrypt('ТОТ, КТО ХОЧЕТ ЕСТЬ ЯЙЦА, ДОЛЖЕН ПРИМИРИТЬСЯ С КУДАХТАНЬЕМ.', al_plaifair,'СЕНТЯБРЬ')

'СФ ЕК ФЕ ФВ ПЦ ЧС НЯ НЕ ЕВ НЛ УД ПШ БВ ХЖ ЗС ЕУ АЖ ЖК АЖ ЕВ ЕБ ТЖ ЦА ГУ НВ ЕА БЗ СЪ ФЪ'

In [90]:
text = prepare(test_text().upper().replace('-', '—').replace('X', '').replace('I', ''))

In [103]:
plaifair_encrypt(text, al_plaifair,'БЛИНДАЖ')[:150]

'ЕР ИН ЕР НД КА ДП ШК ПА ТМ ДХ ЗР ТК ЫТ ПН ХД ЛЦ ЛР ТМ ЮО ЛР КЛ РС ЛГ ЛР ТЮ ОК ХЛ ВФ ТП ЧО ЖШ ЯЗ РЛ ЭР РХ ДЫ ТК ЧЖ БИ ЭЛ ПЛ ОЗ ЮК ДП ВФ ЕБ ПТ ОЯ ГС ШТ '

In [102]:
plaifair_decrypt(plaifair_encrypt(text, al_plaifair,'БЛИНДАЖ'), al_plaifair,'БЛИНДАЖ')[:150]

'НЕЛИНЕИНАЯИСТАТИСТИЧЕСКАЯПРИЧИНФНОСТЪСНОВАПРИВНОСЯТВФИЛОСОФСКУЮКОНЦЕПЦИЮКАУЗАЛЪНОИСВЯЗИСЛОЖНОСТЪ,КОТОРАЯБЫЛАПОТЕРЯНАСПОЯВЛЕНИЕМПОНЯТИЯПОСТОЯННОИКОНЪЮН'